### Introduction

This notebook is about how to do cross-validation once you have decided that oversampling is the right approach for imbalance problem.So  in this notebook i have focused on modeling , You can't fin feat eng , feat selection , hyper parameter optimization and so on.  If you want to play with the process yourself;you can complete the pipeline applying better algorithm(s) and feat eng as well. 

we will be going through the following steps:

- Getting a base model with random forest 
- Oversampling the wrong way: Do oversample, then oof cross-validate. Sounds fine, but results are overly optimistic.
- Oversampling the right way: Make SMOTE a part of our oof cross validation and compare recall and confusion matrix

also i used answer.npy , the target values of test data to evaluate unseen data for models.

Our goal will be to find a classifier with a good recall (i.e. we want our classifier to find as many positive cases as it can). So i didn't apply method to boost roc-auc for example. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file \\I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

from scipy import stats
from scipy.stats import norm, skew

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, GroupKFold, GridSearchCV, StratifiedKFold

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsRegressor, KernelDensity, KDTree
from sklearn.metrics import *

from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler, SMOTENC, SVMSMOTE
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.under_sampling import ClusterCentroids, TomekLinks, NearMiss, RandomUnderSampler, AllKNN, CondensedNearestNeighbour,\
                                        NeighbourhoodCleaningRule, OneSidedSelection, RepeatedEditedNearestNeighbours, InstanceHardnessThreshold



import sys, os
import random 

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
from IPython.display import display


def set_seed(seed=4242):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed()

### Loading the data

In [ ]:
train  = pd.read_csv('/kaggle/input/imbalanced-data-practice/aug_train.csv')
test  = pd.read_csv('/kaggle/input/imbalanced-data-practice/aug_test.csv')
answers = np.load('/kaggle/input/answer/answer.npy')
print(train.shape, test.shape)
display(train.head())
display(test.head())

In [ ]:
train.info()

In [ ]:
train.describe(include=['O'])

### Categories

In [ ]:
cats = [c for c in train.columns if train[c].dtypes =='object']
print('Categories', cats)

nums = [c for c in train.columns if c not in cats]
print('Numerics', nums)

In [ ]:
for c in cats:
    le=LabelEncoder()
    le.fit(list(train[c].astype('str')) + list(test[c].astype('str')))
    train[c] = le.transform(list(train[c].astype(str))) 
    test[c] = le.transform(list(test[c].astype(str))) 
train.head()

In [ ]:
del train['id']
del test['id']

target = train.pop('Response')

train.shape, test.shape

In [ ]:
plt.figure(figsize=(15, 12))
sns.heatmap(pd.concat([train, target], axis=1).corr(),annot=True , cmap='vlag') 

In [ ]:
ax = sns.countplot(x = target ,palette="Set2")
sns.set(font_scale=1.5)
ax.set_xlabel(' ')
ax.set_ylabel(' ')
fig = plt.gcf()
fig.set_size_inches(10,5)
ax.set_ylim(top=700000)
for p in ax.patches:
    ax.annotate('{:.2f}%'.format(100*p.get_height()/len(target)), (p.get_x()+ 0.3, p.get_height()+10000))

plt.title('Distribution of Target')

plt.ylabel('Frequency [%]')
plt.show()

### Resampling Methods

you can find several resources and kaggle notebooks for understanding resampling methods  and fundamentals, but the summary is that:
    
>*The main motivation behind the need to preprocess imbalanced data before we feed them into a classifier is that typically classifiers are more sensitive to detecting the majority class and less sensitive to the minority class.* *Thus, if we don't take care of the issue, the classification output will be biased, in many cases resulting in always predicting the majority class. Many methods have been proposed in the past few years to deal with imbalanced data.*



![](https://miro.medium.com/max/725/0*FeIp1t4uEcW5LmSM.png)

#### SMOTE

In SMOTE (Synthetic Minority Oversampling Technique) we synthesize elements for the minority class, in the vicinity of already existing elements.
At a high level, SMOTE creates synthetic observations of the minority class [0] by:

- Finding the k-nearest-neighbors for minority class observations (finding similar observations)
- Randomly choosing one of the k-nearest-neighbors and using it to create a similar, but randomly tweaked, new observation.

![](https://miro.medium.com/max/734/0*_XpwY9GznmejI4WN.png)

#### Tomek Links

In this algorithm, we end up removing the majority element from the Tomek link, which provides a better decision boundary for a classifier.

![](https://miro.medium.com/max/798/0*YWVxE7SbWKnTnbZi)

You can find an overview of most popular methods and advantage and disadvantage here:

[Imbalanced Data : How to handle Imbalanced Classification Problems](https://www.analyticsvidhya.com/blog/2017/03/imbalanced-data-classification/)

### Base Model : Original Data

Lets look how Random Forest models insurance imalanced data. As performance metrics I will use recall and ConfusionMatrix. 

*Note: The goal of your modeling in practical projects could be specified by the business. here our goal is better TP* 


In [ ]:
score_auc = []
score_recall = []
oof_rf = np.zeros(len(train))
pred_rf = np.zeros(len(test))

folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for fold_ , (train_ind, val_ind) in enumerate(folds.split(train, target)):
    print('fold:', fold_, '  - Starting ...')
    trn_data, val_data = train.iloc[train_ind], train.iloc[val_ind]
    y_train, y_val = target.iloc[train_ind], target.iloc[val_ind]
    
    rf = RandomForestClassifier(n_estimators=150, max_depth=5, criterion='gini', max_features=0.8, n_jobs= -1, random_state=32)
    rf.fit(trn_data, y_train)
    oof_rf[val_ind] = rf.predict_proba(val_data)[:, 1]
    y = rf.predict_proba(trn_data)[:, 1]
    print('val auc:' , roc_auc_score(y_val, oof_rf[val_ind]))
    print('val recall:' , recall_score(y_val, np.where(oof_rf[val_ind] > 0.5, 1, 0)))
   
    score_auc.append(roc_auc_score(y_val, oof_rf[val_ind]))
    score_recall.append(recall_score(y_val, np.where(oof_rf[val_ind] > 0.5, 1, 0)))
                        
    pred_rf += rf.predict_proba(test)[:, 1]/folds.n_splits
    
print(' Model auc: -------> ', np.mean(score_auc))
print(' Model recall: -------> ', np.mean(score_recall))

In [ ]:
oof_rf_01 = np.where(oof_rf > 0.5, 1, 0)




cf_matrix = confusion_matrix(target, (oof_rf_01)) 
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.4)
plt.style.use('seaborn-poster')
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='vlag_r')

In [ ]:


pred = np.where(pred_rf>0.50, 1, 0)
pred

recall_score(answers, pred)

So as we can boost auc with different techniques but the result of Confusion Matrix is terrible (TP=0.0%) , so we will continue with resampling. 

### **Upsampling the wrong way !**




>*If cross-validation is done on already upsampled data, the scores don't generalize to new data. In a real problem, you should only use the test set ONCE; we are reusing it to show that if we do cross-validation on already upsampled data, the results are overly optimistic and do not generalize to new data (or the test set).*

In [ ]:
smote= SMOTE(sampling_strategy='minority', random_state=2020, k_neighbors=5) # used default imblearn parameters

train_resampled, target_resampled = smote.fit_resample(train, target)
train_resampled.shape, target_resampled.shape, test.shape

In [ ]:
type(train_resampled)

In [ ]:
ax = sns.countplot(x = target_resampled ,palette="vlag")
sns.set(font_scale=1.5)
ax.set_xlabel(' ')
ax.set_ylabel(' ')
fig = plt.gcf()
fig.set_size_inches(10,5)
ax.set_ylim(top=700000)
for p in ax.patches:
    ax.annotate('{:.2f}%'.format(100*p.get_height()/len(target_resampled)), (p.get_x()+ 0.3, p.get_height()+10000))

plt.title('Distribution of Target')

plt.ylabel('Frequency [%]')
plt.show()

In [ ]:
score_auc = []
score_recall = []

oof_rf = np.zeros(len(train_resampled))
pred_rf = np.zeros(len(test))

folds = KFold(n_splits=3, shuffle=True, random_state=42)

for fold_ , (train_ind, val_ind) in enumerate(folds.split(train_resampled, target_resampled)):
    print('fold:', fold_, '  - Starting ...')
    trn_data, val_data = train_resampled.iloc[train_ind], train_resampled.iloc[val_ind]
    y_train, y_val = target_resampled.iloc[train_ind], target_resampled.iloc[val_ind]
    
    rf = RandomForestClassifier(n_estimators=150, max_depth=5, criterion='gini', max_features=0.8, min_samples_split=5, n_jobs= -1, random_state=32)
    rf.fit(trn_data, y_train)
    oof_rf[val_ind] = rf.predict_proba(val_data)[:, 1]
    y = rf.predict_proba(trn_data)[:, 1]
    #print('train:',roc_auc_score(y_train, y),'val :' , roc_auc_score(y_val, oof_rf[val_ind]))
    
    print('val auc:' , roc_auc_score(y_val, oof_rf[val_ind]))
    print('val recall:' , recall_score(y_val, np.where(oof_rf[val_ind] > 0.5, 1, 0)))
    
    
    
    score_auc.append(roc_auc_score(y_val, oof_rf[val_ind]))
    score_recall.append(recall_score(y_val, np.where(oof_rf[val_ind] > 0.5, 1, 0)))
    pred_rf += rf.predict_proba(test)[:, 1]/folds.n_splits
    
print(' Model auc: -------> ', np.mean(score_auc))
print(' Model recall: -------> ', np.mean(score_recall))

In [ ]:
oof_rf_01 = np.where(oof_rf > 0.5, 1, 0)
cf_matrix = confusion_matrix(target_resampled, (oof_rf_01)) 
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.4)
plt.style.use('seaborn-poster')
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='vlag_r')

In [ ]:


pred = np.where(pred_rf>0.50, 1, 0)

recall_score(answers, pred)

#### *So our model is not generalized: cv recall = 0.94535924158125 , test recall= 0.9087221095334685*

To see why it happened, Let's say every data point from the minority class is copied 6 times before making the splits. If we did a 3-fold validation, each fold has (on average) 2 copies of each point! If our classifier overfits by memorizing its training set, it should be able to get a perfect score on the validation set! Our cross-validation will choose the model that overfits the most. We see that CV chose the deepest trees it could!

Instead, we should split into training and validation folds. Then, on each fold:

>1.Oversample the minority class
>
>2.Train the classifier on the training folds
>
>3.Validate the classifier on the remaining fold




### **Upsampling the Right way !**

In [ ]:
smote= SMOTE(sampling_strategy='minority', k_neighbors=5) # used default imblearn parameters


tml = TomekLinks()

In [ ]:
score_auc = []
score_recall = []


oof_rf = np.zeros(len(train))
pred_rf = np.zeros(len(test))

folds = KFold(n_splits=3, shuffle=True, random_state=42)

for fold_ , (train_ind, val_ind) in enumerate(folds.split(train, target)):
    print('fold:', fold_, '  - Starting ...')
    trn_data, val_data = train.iloc[train_ind], train.iloc[val_ind]
    y_train, y_val = target.iloc[train_ind], target.iloc[val_ind]
    
    train_upsample, y_upsample = smote.fit_resample(trn_data, y_train)
    rf = RandomForestClassifier(n_estimators=150, max_depth=5, criterion='gini', max_features=0.8, min_samples_split=5, n_jobs= -1, random_state=32)
    rf.fit(train_upsample, y_upsample)
    
    oof_rf[val_ind] = rf.predict_proba(val_data)[:, 1]
    
    y = rf.predict_proba(train_upsample)[:, 1]
    
    print('val auc:' , roc_auc_score(y_val, oof_rf[val_ind]))
    print('val recall:' , recall_score(y_val, np.where(oof_rf[val_ind] > 0.5, 1, 0)))
    
    score_auc.append(roc_auc_score(y_val, oof_rf[val_ind]))
    score_recall.append(recall_score(y_val, np.where(oof_rf[val_ind] > 0.5, 1, 0)))
            
    
    pred_rf += rf.predict_proba(test)[:, 1]/folds.n_splits
    
print(' Model auc: -------> ', np.mean(score_auc))
print(' Model recall: -------> ', np.mean(score_recall))


In [ ]:

oof_rf_01 = np.where(oof_rf > 0.5, 1, 0)

cf_matrix = confusion_matrix(target, (oof_rf_01)) 
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.4)
plt.style.use('seaborn-poster')
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='vlag_r')

In [ ]:
pred_rf

In [ ]:
pred = np.where(pred_rf>0.50, 1, 0)
recall_score(answers, pred)

CV recall = 0.9057 , test recall = 0.9088

So not only  we have very fit model(cv and test recall are very consistent) but also we have better confusion matrix and recall in comparision with first model, and model upsampling the wrong way is completely overfit.

**Conclusion**

1. Considering appropriate metric is a key and also depends on your business use case and its goal if we want more TPs so auc wont be a good chice.
2. Resampling is one of the techniques to handle imbalanced problem if we apply resampling the right way. some time othe techniques such as class weight or applying better algoritms or ensemble modeling could help more. 
3. We just focused on modeling,  feature engineering and selection in many cases could boost the performance regarding recall and confusion  matrix.